In [1]:
from gabor_feature import get_feature_vector
import cv2
import os
from collections import *
import numpy as np
from sklearn.cluster import KMeans

_k = 8
_n = 4
_samples = 16
_w0 = 0.13
_folder_views = "images/"
_n_clusters = 10

def getAllViews():
    l = os.listdir(_folder_views)
    all_views = []
    for i in l:
        all_views.append(_folder_views + i)
    return all_views

In [2]:
img = cv2.imread("sketches/888.png", cv2.COLOR_RGB2GRAY)
basic_views = ["images/m508_outfile_10.jpg"]

In [3]:
import tqdm

In [4]:
features = []
features_per_objectview = []
names = []
basic_views = getAllViews()
for basic_view in tqdm.tqdm(basic_views[:11]):
    sketch = cv2.Canny(cv2.imread(basic_view, 0), 100, 200)
    if np.sum(sketch) == 0:
        continue
    features_objectview = get_feature_vector(sketch, k=_k, n=_n, samples=_samples)
    features.extend(features_objectview )
    features_per_objectview.append(features_objectview)
    names.append(basic_view)
kmeans = KMeans(n_clusters=_n_clusters, random_state=0).fit(features)

100%|██████████| 11/11 [00:08<00:00,  1.32it/s]


In [5]:
def createHistogram(centers):
    h = Counter()
    for i in centers:
        h[i] += 1
    h = dict(sorted(h.items()))
    return h

def createHistograms(kmeans, features_per_objectview):
    histograms = []
    for feature_objectview in features_per_objectview:
        histograms.append(createHistogram(kmeans.predict(feature_objectview)))
    histograms.sort(key=lambda x: [(i, x[i]) for i in x.keys()])
    return histograms

def processSketchHistogram(h,N,f):
    hi_sum = np.sum(list(h.values()))
    print(hi_sum)

    for j in h.keys():    
        h[j] = h[j]/hi_sum*np.log(N/f[j])
    h_sum = np.sum([i**2 for i in h.values()])
    for j in h.keys():
        h[j] = h[j]/h_sum
    print(h)

In [6]:
histograms = createHistograms(kmeans, features_per_objectview)

In [7]:
histograms

[{0: 2, 1: 24, 2: 13, 3: 24, 4: 25, 5: 11, 6: 15, 7: 6, 8: 14, 9: 7},
 {0: 4, 1: 20, 2: 8, 3: 2, 4: 4, 5: 33, 6: 4, 7: 8, 8: 13, 9: 8},
 {0: 4, 1: 27, 2: 26, 3: 18, 4: 3, 5: 26, 6: 15, 7: 30, 8: 22, 9: 12},
 {0: 8, 1: 30, 2: 19, 3: 24, 4: 32, 5: 13, 6: 6, 7: 7, 8: 15, 9: 13},
 {0: 10, 1: 11, 2: 5, 3: 24, 4: 16, 5: 27, 6: 54, 7: 20, 8: 58, 9: 5},
 {0: 14, 1: 4, 2: 15, 3: 19, 4: 14, 5: 21, 6: 16, 7: 20, 8: 24, 9: 18},
 {0: 14, 1: 5, 2: 22, 3: 11, 4: 20, 5: 10, 6: 17, 7: 27, 8: 37, 9: 14},
 {0: 16, 3: 72, 4: 4, 5: 35, 6: 36, 7: 11, 8: 31, 9: 19},
 {0: 18, 1: 6, 2: 15, 3: 3, 4: 11, 6: 4, 7: 13, 8: 18, 9: 25},
 {0: 19, 1: 38, 2: 23, 3: 4, 4: 42, 6: 1, 7: 19, 8: 15, 9: 7},
 {1: 24, 2: 20, 3: 4, 4: 19, 6: 4, 7: 1, 8: 5}]

In [11]:
N = len(histograms)
f = np.zeros(kmeans.n_clusters)
for h in histograms:
    for j in h.keys():
        f[j] = (f[j] + 1)


In [12]:
example = np.random.random([_samples*_samples, _n**2*_k])

In [13]:
#f = f/np.sum(f)

In [14]:
histogram_for_sketch = createHistogram(kmeans.predict(example))
    

In [15]:
histogram_for_sketch

{1: 5, 3: 67, 5: 31, 6: 1, 8: 152}

In [16]:
print(histogram_for_sketch)
processSketchHistogram(histogram_for_sketch, N, f)

{1: 5, 3: 67, 5: 31, 6: 1, 8: 152}
256
{1: 1.2488847146113295, 3: 0.0, 5: 25.871469208354064, 6: 0.0, 8: 0.0}


In [ ]:
class RangedHistograms():
    def __init__(self, length):
        self.histograms = []
        self.max_len = length
        
    def add(self, h):
        if len(self.histograms) == 0:
            self.histograms.append(h)
        else:
            for i in list(h.keys):
                print(i)
            
    def show(self):
        print(self.histograms)
            
t = RangedHistograms(10)
t.add(histograms[0])
t.show()
t.add(histograms[1])